In [1]:
%matplotlib inline
import json 
import matplotlib.pyplot as plt
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import keras 


Using TensorFlow backend.
/home/omid/.conda/envs/iot_new/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Load the data

In [2]:
x_data= [] 
y_data= [] 

with open( '../files/txt/seq_mapping_large.txt' ) as f:
    x_data = f.readlines()

with open( '../files/txt/command_mapping_large.txt' ) as f:
    y_data = f.readlines()
    
    
x_data =[ np.array([ int(y) for y in x.strip().split( ' ') ])   for x in  x_data ] 
y_data =[ x.strip().split(' ') for x in  y_data ] 


In [3]:
np.max( np.concatenate( x_data ))

1272

In [4]:
len(np.unique(  np.concatenate( y_data )))

27

In [5]:
classes  = list(np.unique(  np.concatenate( y_data )))

In [6]:
classes

['Kwikset_10-Button_Deadbolt_activity_hubDisconnected',
 'Kwikset_10-Button_Deadbolt_activity_online',
 'Kwikset_10-Button_Deadbolt_lock_locked',
 'Kwikset_10-Button_Deadbolt_lock_unlocked',
 'Multipurpose_Sensor_acceleration_active',
 'Multipurpose_Sensor_acceleration_inactive',
 'Multipurpose_Sensor_activity_hubDisconnected',
 'Multipurpose_Sensor_activity_online',
 'Multipurpose_Sensor_contact_closed',
 'Multipurpose_Sensor_contact_open',
 'Multipurpose_Sensor_status_closed',
 'Multipurpose_Sensor_status_open',
 'Multipurpose_Sensor_temperature_XXX',
 'Multipurpose_Sensor_threeAxis_XXX',
 'OSRAM_LIGHTIFY_Dimming_Switch_activity_hubDisconnected',
 'OSRAM_LIGHTIFY_Dimming_Switch_activity_online',
 'OSRAM_LIGHTIFY_Dimming_Switch_button_held',
 'OSRAM_LIGHTIFY_Dimming_Switch_button_pushed',
 'SYLVANIA_SMART+_Smart_Plug_switch_off',
 'SYLVANIA_SMART+_Smart_Plug_switch_on',
 'SYLVANIA_Smart_10Y_A19_TW_activity_hubDisconnected',
 'SYLVANIA_Smart_10Y_A19_TW_activity_offline',
 'SYLVANIA_Sma

In [31]:
y_data_categorical = []  

for x in y_data:
    temp = np.zeros( len(classes) )
    for y in x : 
        temp[ classes.index( y ) ] = 1
    y_data_categorical.append( temp )
y_data_categorical = np.vstack(y_data_categorical)

y_data_categorical=y_data_categorical.reshape(y_data_categorical.shape+(1,))



In [44]:
y_data_categorical=y_data_categorical.reshape(y_data_categorical.shape+(1,))

In [45]:
print(y_data_categorical.shape)

(431, 27, 1, 1)


In [46]:
x_data = np.array( x_data) / 1500.0

# Create the Model 

In [47]:

# import numpy
# import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf

In [48]:
x_data_temp = [] 
for x in x_data:
    temp = list(x)
    while len(temp) < 256*256:
        temp.append(0)
    
    x_data_temp.append(np.array( temp).reshape(256,256))

    
x_data_temp = np.array( x_data_temp )
x_data_temp=x_data_temp.reshape(x_data_temp.shape+(1,))

x_data_temp.shape

(431, 256, 256, 1)

In [49]:
np.min([ len(x) for x in x_data_temp ])

256

In [50]:
len(classes) 

27

In [51]:
def baseline_model():
    
    model = keras.Sequential([
    keras.layers.Flatten(input_shape=(13,13)),
    keras.layers.Dense(320, activation=tf.nn.relu),
    keras.layers.Dense(27, activation=tf.nn.softmax)
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    return model

#     # create model
#     model = Sequential()
#     model.add(Dense(320, input_dim=160, activation='relu'))
#     model.add(Dense(len(classes), activation='softmax'))
#     # Compile model
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [77]:
import numpy as np
import os
import tensorflow as tf
from keras.models import Model

from keras.layers import Input, merge, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, Dropout, Conv2DTranspose, UpSampling2D, Lambda
from keras.layers.normalization import BatchNormalization as bn
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import RMSprop
from keras import regularizers
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers.merge import add
from keras.models import load_model
import numpy as np
from keras.regularizers import l2

def unet():
    l2_lambda = 0.0002
    DropP = 0.3
    kernel_size=3

    inputs = Input((256,256,1))
    #Conv3D(filters,(3,3,3),sjfsjf)
    conv1 = Conv2D(16, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda), name='conv1' )(inputs)
    conv1 = bn()(conv1)
    conv1 = Conv2D(16, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv1)
    conv1 = bn()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(DropP)(pool1)

    conv2 = Conv2D(32, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda), name='conv2' )(pool1)
    conv2 = bn()(conv2)
    conv2 = Conv2D(32, (kernel_size, kernel_size), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv2)
    conv2 = bn()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(DropP)(pool2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) , name='conv3')(pool2)
    conv3 = bn()(conv3)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv3)
    conv3 = bn()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(DropP)(pool3)
    
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda), name='conv4')(pool3)
    conv4 = bn()(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv4)
    conv4 = bn()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(DropP)(pool4)

    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda), name='conv5' )(pool4)
    conv5 = bn()(conv5)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_lambda) )(conv5)
    conv5 = bn()(conv5)
    
    up6 = concatenate([Conv2DTranspose(256,(2, 2), strides=(2, 2), padding='same')(conv5), conv4],name='up6', axis=3)
    up6 = Dropout(DropP)(up6)
    output = Conv2D(1, (5, 5), activation='softmax', padding='same', name='output')(up6)
       

    #output = Dense(27,  activation='softmax',name='output')(conv6)
    
    model = Model(inputs=[inputs], outputs=[output])
    model.compile(optimizer=Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [78]:
#model = baseline_model()
#report = model.fit( x=x_data_temp , y=y_data_categorical , batch_size=35, epochs=12000 )

In [75]:
# x_data_temp=x_data_temp.reshape(x_data_temp.shape+(1,))

model2 = unet()
model2.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_154 (Conv2D)             (None, 256, 256, 16) 160         input_13[0][0]                   
__________________________________________________________________________________________________
batch_normalization_145 (BatchN (None, 256, 256, 16) 64          conv2d_154[0][0]                 
__________________________________________________________________________________________________
conv2d_155 (Conv2D)             (None, 256, 256, 16) 2320        batch_normalization_145[0][0]    
__________________________________________________________________________________________________
batch_norm

In [79]:
report2 = model2.fit( x=x_data_temp , y=y_data_categorical , batch_size=35, epochs=1 )

ValueError: Error when checking target: expected output to have shape (32, 32, 1) but got array with shape (27, 1, 1)

In [42]:
# x_data_temp=x_data_temp.reshape(x_data_temp.shape+(1,))
print(x_data_temp.shape , y_data_categorical.shape)

(431, 256, 256, 1) (431, 27, 1)


In [ ]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
results = cross_val_score(estimator, x_data_temp, y_data, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
tf.__version__

In [ ]:
for i in range(16):
    print( i , i**2 )

In [ ]:
a= [1,2,3,4]
np.array(a).reshape( 2,2 )